In [1]:
# ⬇️ 1‒2 分ほど
!pip -q install "diffusers[torch]>=0.28.0" transformers accelerate peft torchmetrics einops safetensors
!pip -q install --upgrade torchmetrics[image] torch-fidelity
!pip -q install --upgrade torch-fidelity           # ★ 追加で明示インストール

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# データと重みの保存先（★ 必要に応じて書き換え）
GDRIVE_DATA_DIR = "/content/drive/MyDrive/AffectNet"
GDRIVE_SAVE_DIR = "/content/drive/MyDrive/expr_edit_model"
!mkdir -p "$GDRIVE_SAVE_DIR"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import ast, torch, pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

IMG_RES = 256           # Stable Diffusion 1.x 系のデフォルト

# ── AU ベクトルの前処理 ──────────────────────────
def scale_au(v):        # 入力 0‑5 → ‑1…1 に線形スケール
    v = torch.tensor(v, dtype=torch.float32)
    return (v / 2.5) - 1.0

# ── 画像前処理 ──────────────────────────────────
img_tf = transforms.Compose([
    transforms.Resize((IMG_RES, IMG_RES), interpolation=Image.BICUBIC),
    transforms.ToTensor(),                      # 0‒1
    transforms.Normalize([0.5]*3, [0.5]*3)      # ‑1‒1
])

class AUPairs(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)
        self.paths = df['image_path'].tolist()
        self.au    = [ast.literal_eval(s) for s in df['au_vector']]
    def __len__(self): return len(self.paths)
    def __getitem__(self, idx):
        img = img_tf(Image.open(self.paths[idx]).convert("RGB"))
        au  = scale_au(self.au[idx])
        return {"pixel_values": img, "au_vec": au}

train_ds = AUPairs(f"{GDRIVE_DATA_DIR}/train_pairs.csv")
test_ds  = AUPairs(f"{GDRIVE_DATA_DIR}/test_pairs.csv")

BATCH = 4
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=4, pin_memory=True)


In [4]:
import torch.nn as nn, torch.nn.functional as F, math
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from peft import LoraConfig, get_peft_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype  = torch.float32

# --- 2‑1. Stable Diffusion v1‑5 の VAE & UNet ----------------
vae  = AutoencoderKL.from_pretrained(
        "runwayml/stable-diffusion-v1-5", subfolder="vae",
        torch_dtype=dtype
      ).to(device)

unet = UNet2DConditionModel.from_pretrained(
        "runwayml/stable-diffusion-v1-5", subfolder="unet",
        torch_dtype=dtype
      ).to(device)

unet.enable_gradient_checkpointing()      # VRAM 節約

# --- 2‑2. UNet を LoRA 化（4‑8 ランクで軽量学習） -----------
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],  # cross‑attention 層
    lora_dropout=0.05,
    bias="none"
)
unet = get_peft_model(unet, lora_cfg)
unet.train()

# ───────────────────────────────────────────────
# 2‑3. AU → 768 埋め込みモジュール  ★ 修正版
# ───────────────────────────────────────────────
import torch.nn as nn

# データセットから実際の AU 次元を取得
AU_DIM = len(train_ds[0]["au_vec"])     # 例: 35

class AUEmbed(nn.Module):
    def __init__(self, in_dim=AU_DIM, out_dim=768):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.SiLU(),
            nn.Linear(512, out_dim)
        )
    def forward(self, au):
        return self.mlp(au)[:, None]     # (B,1,768)

au_embed = AUEmbed().to(device, dtype)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from accelerate import Accelerator
from tqdm.auto import tqdm
import torch.nn.functional as F, torch
import torch

# ── 3‑0. Accelerator の初期化 ──
precision = "fp16" if dtype == torch.float16 else None
accelerator = Accelerator(mixed_precision=precision)
device      = accelerator.device            # 以降はこれで統一

# ── 3‑1. DataLoader / モデルを accelerator に渡す ───────────────
unet, au_embed, train_dl = accelerator.prepare(unet, au_embed, train_dl)

noise_sched = DDPMScheduler(num_train_timesteps=1000, beta_schedule="squaredcos_cap_v2")

opt = torch.optim.AdamW(
    list(unet.parameters()) + list(au_embed.parameters()),
    lr=1e-4, weight_decay=1e-2
)

EPOCHS  = 1          # 動作確認なら 1 epoch
DROP_P  = 0.1
GRAD_ACCUM_STEPS = 4        # 4 step で 1 回 optimizer.step()
MAX_GRAD_NORM = 1.0          # 1.0～5.0 が一般的

for _ in range(EPOCHS):
    pbar = tqdm(train_dl, disable=not accelerator.is_local_main_process)
    for step, batch in enumerate(pbar):
        # ---------------------- 型とデバイスを統一 ----------------------
        pix = batch["pixel_values"].to(device, dtype=dtype)     # ⭐ ここを修正
        au  = batch["au_vec"].to(device,   dtype=dtype)         # ⭐ ここも

        # ---------- VAE で latent へ（VAE は事前学習なので勾配不要） ----
        with torch.no_grad():
            lat = vae.encode(pix).latent_dist.sample() * 0.182

        # ---------- DDPM ノイズ付与 ------------------------------------
        noise     = torch.randn_like(lat)
        timesteps = torch.randint(
            0, noise_sched.config.num_train_timesteps,  # ← FutureWarning 回避
            (lat.size(0),), device=device
        ).long()
        noisy_lat = noise_sched.add_noise(lat, noise, timesteps)

        # ---------- Classifier‑Free Guidance ---------------------------
        cond_mask = (torch.rand(lat.size(0), device=device) < DROP_P).float().unsqueeze(1)
        au_in     = au * (1 - cond_mask)        # 無条件パス用の 0‑vector
        h         = au_embed(au_in)

        # ---------- UNet 予測 & 損失 -----------------------------------
        model_pred = unet(noisy_lat, timesteps, encoder_hidden_states=h).sample
        loss = F.mse_loss(model_pred.float(), noise.float(), reduction="mean")
        loss = loss / GRAD_ACCUM_STEPS

        if torch.isnan(loss):
            print("⚠️  Nan detected ‑‑ skipping step")
            continue

        accelerator.backward(loss)

        # ★ 追加: 勾配をノルムで制限
        accelerator.clip_grad_norm_(unet.parameters(), MAX_GRAD_NORM)

        if (step + 1) % GRAD_ACCUM_STEPS == 0:
            opt.step(); opt.zero_grad()

        if accelerator.is_local_main_process:
            pbar.set_description(f"loss: {loss.item():.4f}")


  0%|          | 0/2796 [00:00<?, ?it/s]

In [ ]:
# LoRA 部分
unet.save_pretrained(GDRIVE_SAVE_DIR, safe_serialization=True)
# AU Embedder
torch.save(au_embed.state_dict(), f"{GDRIVE_SAVE_DIR}/au_embed.pth")
print("✅  saved to:", GDRIVE_SAVE_DIR)


In [ ]:
from diffusers import DDIMScheduler, StableDiffusionImg2ImgPipeline, UNet2DConditionModel
from peft import PeftModel
from torchvision.transforms.functional import to_pil_image
import torch, gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype  = torch.float32           # 学習を FP32 にしたので合わせる
IMG_RES = 256

# ① ベース UNet
base_unet = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="unet", torch_dtype=dtype
).to(device)

# ② LoRA アダプタを当てて merge
unet = PeftModel.from_pretrained(base_unet, GDRIVE_SAVE_DIR)
unet = unet.merge_and_unload()          # ← LoRA を焼き付けて普通の UNet に
unet.to(device, dtype)

# ③ VAE をロード（FP32）
vae = AutoencoderKL.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="vae", torch_dtype=dtype
).to(device)

# ④ AU Embedder をロード
au_embed = AUEmbed().to(device, dtype)
au_embed.load_state_dict(torch.load(f"{GDRIVE_SAVE_DIR}/au_embed.pth", map_location="cpu"))

# ⑤ スケジューラ
scheduler = DDIMScheduler.from_pretrained(
    "runwayml/stable-diffusion-v1-5", subfolder="scheduler"
)

# ⑥ パイプライン
pipe = StableDiffusionImg2ImgPipeline(
    vae=vae,
    text_encoder=None,
    tokenizer=None,
    unet=unet,
    scheduler=scheduler,
    safety_checker=None,
    feature_extractor=None
).to(device, dtype)

pipe.enable_attention_slicing()   # VRAM 節約（推論時のみ FP32 でも収まる）
gc.collect(); torch.cuda.empty_cache()


In [ ]:
# ---------- 17 強度 → 35 dim に拡張 ----------------------------
AU_DIM = 35          # 学習時の次元

def build_au_vector(intensities):
    if len(intensities) == AU_DIM:
        return intensities
    if len(intensities) != 17:
        raise ValueError("AU list must be 17 or 35 dim")

    presence = [1 if x > 0 else 0 for x in intensities]  # 17
    presence.append(1 if any(intensities) else 0)        # AU45_c
    return intensities + presence                        # 17 + 18 = 35

# ---------- スケール 0‑5 → -1..1 -------------------------------
def scale_au(v):
    v = torch.tensor(v, dtype=dtype)
    return (v / 2.5) - 1.0

# ---------- 推論関数 -------------------------------------------
def edit_expr(img_pil, target_au17,
              strength=0.6, guidance=7.5, steps=50):
    # 1) AU ベクトルを 35 次元 & スケール
    au35 = build_au_vector(target_au17)
    with torch.no_grad():
        prompt_embeds = au_embed(scale_au(au35).unsqueeze(0).to(device))  # (1,1,768)

    # 2) ガイダンス用の negative_prompt_embeds (= 0 ベクトル)
    neg_embeds = torch.zeros_like(prompt_embeds)

    # 3) 画像を 256×256 にリサイズして推論
    result = pipe(
        image              = img_pil.resize((IMG_RES, IMG_RES)),
        prompt_embeds      = prompt_embeds,
        negative_prompt_embeds = neg_embeds,
        num_inference_steps= int(strength * steps),
        guidance_scale     = guidance,
    )
    return result.images[0]

# ---------- テスト ----------------------------------------------
from PIL import Image, ImageOps, ImageDraw, ImageFont
import gc

sample_img  = Image.open(train_ds.paths[0]).convert("RGB")
target_au17 = [0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0]  # AU05: upper‑lid raiser
edited_img = edit_expr(sample_img, target_au17)

# すでに生成済みの edited_img と sample_img を使う
combined = Image.new("RGB", (IMG_RES*2, IMG_RES))
combined.paste(sample_img.resize((IMG_RES, IMG_RES)), (0, 0))
combined.paste(edited_img.resize((IMG_RES, IMG_RES)), (IMG_RES, 0))

draw = ImageDraw.Draw(combined)
font = ImageFont.load_default()
draw.text((10, 10), "Input", fill=(255, 255, 255), font=font)
draw.text((IMG_RES+10, 10), "Edited", fill=(255, 255, 255), font=font)

display(combined)

# メモリ後片付け
gc.collect(); torch.cuda.empty_cache()


In [ ]:
# ❶ インストールは済んでいる前提
# !pip install -q --upgrade torchmetrics[image] torch-fidelity

import importlib, sys

# ❷ すでに読み込まれていた torchmetrics.image.fid をリロード
if 'torchmetrics.image.fid' in sys.modules:
    importlib.reload(sys.modules['torchmetrics.image.fid'])

# ❸ ふたたびインポートすると依存チェックが通る
from torchmetrics.image.fid import FrechetInceptionDistance
from random import sample
from torchvision.transforms.functional import to_pil_image
from tqdm.auto import tqdm
import torchvision.transforms as T

# ── FID 計算（CPU 推奨）───────────────────────────────
fid = FrechetInceptionDistance(feature=2048, normalize=True).to("cpu")

N = 200
indices = sample(range(len(test_ds)), N)

for idx in tqdm(indices):
    data      = test_ds[idx]
    img_real  = (data["pixel_values"] * 0.5 + 0.5).clamp(0, 1)
    fid.update(img_real.unsqueeze(0), real=True)

    gen_pil   = edit_expr(to_pil_image(img_real), data["au_vec"])
    gen_tensor = T.ToTensor()(gen_pil).unsqueeze(0)
    fid.update(gen_tensor, real=False)

print("FID:", fid.compute().item())
